In [1]:
from Liputan6 import Liputan6 #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [5]:
# health
obj = Liputan6()
end_date = date(2018, 8, 8)
start_date = date(2018, 1, 19)
init_page = 3
list_cat = [['news', 'news'], ['otomotif', 'otomotif'], ['properti', 'properti'],['bisnis','bisnis'],['showbiz','showbiz'],
           ['bola','bola'],['tekno','tekno'],['fashion-beauty','fashion-beauty'],['lifestye','lifestye']]
init_cat = list_cat[0:]

In [ ]:
for single_date in daterange(start_date, end_date):
    for cat in init_cat:
        clear_output()
        print(single_date.strftime("%Y-%m-%d"))
        print(cat[0])
        obj.getAllBerita([], init_page, cat[0], cat[1], single_date.strftime("%Y/%m/%d"))
        init_page = 1
    init_cat = list_cat

2018-01-19
showbiz
page  1
https://www.liputan6.com/showbiz/indeks/2018/01/19?page=1
Insert berita  Cerita Rakyat Mengerikan Ini Cocok Dijadikan Film Horor
masuk
Insert berita  Ada Penampakan Mr P di Kostum Ant-Man and the Wasp
masuk
Insert berita  Raffi Ahmad Sebut Tarif Ayu Ting Ting Mahal
masuk
Insert berita  Taylor Swift Tak Masuk Grammy Awards 2018, Jay Z Mendominasi
masuk
Insert berita  Prosesi Siraman Selesai, Ardina Rasti Katakan Ini pada Ibunda
masuk
Insert berita  Orang Ketiga: Yuni Tinggal di Rumah Afifah - Aris
masuk
Insert berita  Ungkap Alasan Enggan Hapus Tato, Virgoun Justru Semakin Dipuji
masuk
Insert berita  Anak Langit: Ali Diajak Tinggal di Panti, Nyawa Hiro Dalam Bahaya
masuk
Insert berita  Tatonya Tuai Reaksi Keras, Shafa Harris: Ini Karya Seni
masuk
Insert berita  Dude Harlino Angkat Wisata Yogyakarta Lewat Gim
masuk
Insert berita  Kenakan Hijab Selepas JKT48, Nabilah Ayu Dijuluki Bidadari
masuk
Insert berita  Cerita Cinta Josh Duhamel dan Jennifer Garner Mulai T